In [1]:
from model import atp_graph, losses
from Tutorials.helper import batcher
from data import synthetic_data_gen, feature_extractor
import keras
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import pandas as pd
import tensorflow as tf
import IPython
import time
import os

from model import atp_pipeline

In [2]:
pd_array = pd.read_csv("datasets/weather.csv")

In [3]:
data = np.array(pd_array)

In [4]:
data[:,0] = np.linspace(-1,1,data.shape[0])

In [5]:
data = data.astype("float32")

In [6]:
training_data = data[:int(0.7*data.shape[0])]
val_data = data[int(0.7*data.shape[0]):int(0.8*data.shape[0])]
test_data = data[int(0.9*data.shape[0]):]

In [7]:
#scale

training_data_scaled = (training_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)
val_data_scaled = (val_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)
test_data_scaled =  (test_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)

In [8]:
save_dir = "/users/omernivron/Documents/ATP/weights/forecasting/weather"

In [10]:
tf.random.set_seed(442)
opt = tf.keras.optimizers.Adam(3e-4)
run=3; heads=2
atp_model = atp_pipeline.atp_pipeline()
EPOCHS = 2

Metal device set to: Apple M1 Max


2023-04-30 17:20:59.900612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-30 17:20:59.900955: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [11]:
seq_l = 96+192;
num_batches = 500
batch_s=20
mini = 50000
n_C_te =5

In [13]:
step = 1
tr_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [14]:
for epoch in range(2):
    print("epoch: ",+epoch)
    start = time.time()
    for batch_n in range(10):
         
        x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
        n_C = int(np.random.choice(np.arange(2, 20), 1))
        n_T = 288 - n_C
        y_tr = y
        t_tr = x
        _,_, nll_pp, msex = tr_step(atp_model, opt, t_tr,y_tr,n_C,n_T, training=True)

        if ((batch_n % 2 == 0)):
            
            n_C = 10
            n_T = 200 - n_C
            t_te,y_te = batcher(val_data_scaled[:,:1], val_data_scaled[:,-2:],batch_s = 100)
            μ, log_σ = atp_model([t_te, y_te, 10, 200, False])
            _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
            
        if nll_pp_te < mini:
            mini = nll_pp_te
            IPython.display.clear_output(wait=True)
            print("epoch {} batch {} validation lik pp: {}".format(epoch, batch_n, nll_pp)) 
            manager.save()
            step += 1
            ckpt.step.assign_add(1)
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

epoch 1 batch 8 validation lik pp: 0.6718450784683228
Time taken for 1 epoch: 28.91314935684204 secs



In [17]:
nll_list = []; mse_list = []
num_epochs = 2;num_batches = 10; test_batch_s = 100
num_runs = 5 ### used to have a std over likelhood results

for run in range(5):
    sum_mse_tot = 0; sum_nll_tot = 0
    tf.random.set_seed(run)
    for epoch in range(num_epochs):
        print("epoch: ",+epoch)
        start = time.time()
        for batch_n in range(num_batches):
            x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
            n_C = int(np.random.choice(np.arange(2, 20), 1))
            n_T = 288 - n_C
            _,_, nll_pp, msex = tr_step(atp_model, opt, x,y,n_C,n_T, training=True)
    for _ in range(test_data_scaled.shape[0] // test_batch_s):
        n_C = 10
        n_T = 200 - n_C
        t_te,y_te = batcher(test_data_scaled[:,:1], test_data_scaled[:,-2:], batch_s = test_batch_s)
        μ, log_σ = atp_model([t_te,y_te,10,200, False])
        _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ[:, n_C:], log_σ[:, n_C:])
        sum_nll_tot += sum_nll # each sequence is length 190 times 100 seq. per batch 
        sum_mse_tot += sum_mse
        
    nllx =  sum_nll_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
    msex =  sum_mse_tot / (n_T * test_batch_s * (test_data_scaled.shape[0] // test_batch_s))
    
        
    nll_list.append(nllx.numpy())
    mse_list.append(msex.numpy())
np.save(save_dir + '/nll_list.npy', nll_list)    
np.save(save_dir + '/mse_list.npy', mse_list)    

epoch:  0


2023-04-30 16:59:28.722885: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:33.194323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:38.228749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:42.366600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:48.822430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 16:59:53.695908: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:04.466689: I tensorflow/core/grappler/optimizers/cust

epoch:  1


2023-04-30 17:00:23.658776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:30.318846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:40.900534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:00:47.548930: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  0


...

2023-04-30 17:06:16.750343: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:28.825705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:39.023297: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:06:45.185655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  0


2023-04-30 17:07:09.548425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:14.081493: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:21.297863: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:33.094102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:43.536016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:07:53.878573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:05.723638: I tensorflow/core/grappler/optimizers/cust

epoch:  1


2023-04-30 17:08:20.023947: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:34.902275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:08:46.535644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:09:00.301088: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 17:09:15.021889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
from model import testing

In [ ]:
testing.testing_multiple_times([training_data_scaled, test_data_scaled], save_dir)

Metal device set to: Apple M1 Max
epoch:  0


2023-04-30 20:21:57.231179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-30 20:21:57.231429: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-04-30 20:21:59.990918: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-30 20:21:59.991420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:02.881981: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:05.996027: I tensorflow/core/grappler/o

2023-04-30 20:22:15.302552: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2023-04-30 20:22:18.807729: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


epoch:  1


2023-04-30 20:22:22.502387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:26.338168: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:30.944002: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:35.810029: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:39.532378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-04-30 20:22:44.754942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
